In [ ]:
import os
from typing import Any

from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts.base import FormatOutputType

load_dotenv()

In [3]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
messages = [
{"role": "user", "content": "介绍下你自己"}
]

llm = ChatOpenAI(
        model="gpt-3.5-turbo:free",
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0 # 温度是指的模型输出结果的随机性。分数越高，输出结果越随机。
   )
llm.invoke(messages)

AIMessage(content='你好，我是一个语言模型AI助手，可以回答各种问题并与用户进行对话。我可以提供信息、解决问题、进行闲聊等。如果你有任何问题或需要帮助，请随时告诉我。我会尽力帮助你。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 15, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CckqGVq84MPElPJ7AtmQmuvzxTcME', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8a487009-03cf-408f-985a-8052d03e8462-0', usage_metadata={'input_tokens': 15, 'output_tokens': 80, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## PromptTemplate

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
prompt.format(name="算命大师",county="法国",sex="女孩")

'你是一个算命大师,帮我起1个具有法国特色的女孩名字'

## ChatPromptTemplate

In [6]:
# 对话模板具有结构
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个起名大师. 你的名字叫{name}."),
    ("human", "你好{name},你感觉如何？"),
    ("ai", "你好！我状态非常好!"),
    ("human", "你叫什么名字呢?"),
    ("ai", "你好！我叫{name}"),
    ("human", "{user_input}"),
])

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢?")

[SystemMessage(content='你是一个起名大师. 你的名字叫陈大师.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好陈大师,你感觉如何？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好！我状态非常好!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你叫什么名字呢?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='你好！我叫陈大师', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你的爸爸是谁呢?', additional_kwargs={}, response_metadata={})]

In [11]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage

# 直接创建消息
sy = SystemMessage(
  content="你是一个起名大师",
  additional_kwargs={"大师姓名": "陈瞎子"}
)

hu = HumanMessage(
  content="请问大师叫什么?"
)
ai = AIMessage(
  content="我叫陈瞎子"
)
[sy,hu,ai]

[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}, response_metadata={}),
 HumanMessage(content='请问大师叫什么?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫陈瞎子', additional_kwargs={}, response_metadata={})]

## ChatMessagePromptTemplate


In [ ]:
from langchain_core.prompts import AIMessagePromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者",template=prompt)
chat_message_prompt.format(subject="原力")

## 自定义模板

In [2]:
# 函数大师，根据函数名称，查找函数代码，并给出中文代码说明

from langchain_core.prompts.string import StringPromptTemplate
from typing import Any
# 简单的函数作为示例
def hello_world(abc):
    print("Hello, world!")
    return abs;

PROMPT = """
    你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
    函数名称: {function_name}
    源代码:
    {source_code}
    代码解释:
"""
import inspect

def get_source_name(function_name):
    # 获取源代码
    return inspect.getsource(function_name)

# 自定义模板clss
class CustomPromptTemplate(StringPromptTemplate):

    def format(self, **kwargs: Any) -> str:
        # 获得源代码
        source_code = get_source_name(kwargs["function_name"])

        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt


a = CustomPromptTemplate(input_variables=["function_name"])
pm = a.format(function_name=hello_world)
print(pm)

from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model = "gpt-3.5-turbo:free",
    base_url = os.getenv("OPENAI_BASE_URL"),
    api_key = os.getenv("OPENAI_API_KEY"),
    temperature = 0
)
msg = llm.invoke(pm)
print(msg)


    你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
    函数名称: hello_world
    源代码:
    def hello_world(abc):
    print("Hello, world!")
    return abs;

    代码解释:

content='函数名称: hello_world\n\n源代码:\n```python\ndef hello_world(abc):\n    print("Hello, world!")\n    return abs;\n```\n\n中文解释:\n这是一个名为hello_world的函数，它接受一个参数abc。函数的功能是打印出"Hello, world!"并返回abs。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 97, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CcoFRmSVNiR0UPdGivXj1AFuv2FGp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--0104052c-a901-497c-9768-f34ef261f011-0' usage_metadata={'input_tok

## 使用jinja2

In [3]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
from langchain_core.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template,
                                      template_format="jinja2")
prompt.format(name="狗剩", what="高兴")

'给我讲一个关于狗剩的高兴故事'

## 文件管理prompt

In [6]:
from langchain_core.prompts import load_prompt

prompt1 = load_prompt("simple_prompt.yaml")
print(prompt1)

prompt2 = load_prompt("simple_prompt.json")
print(prompt2)

input_variables=['name', 'what'] input_types={} partial_variables={} template='给我讲一个关于{name}的{what}故事'
input_variables=['name', 'what'] input_types={} partial_variables={} template='给我讲一个关于{name}的{what}故事'
